In [5]:
import time
from selenium import webdriver
import urllib
from urllib.request import Request,urlopen

## Scraping 
#### https://www.gsmarena.com/
Scraping all the Samsung tabs present on gsmarena and storing them in Firebase database

In [24]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://www.gsmarena.com/results.php3?sQuickSearch=yes&sName=samsung%20tab')
time.sleep(5)

In [25]:
import requests
from bs4 import BeautifulSoup
import re

In [26]:
data=BeautifulSoup(driver.page_source,'html.parser')

In [27]:
all_tab_urls_list=[]

for getdivtag in data.findAll('div',{'class': 'makers'}):
    for gethref in getdivtag.findAll(['a']):
        all_tab_urls_list.append('https://www.gsmarena.com/' + gethref['href'])


In [28]:
all_tab_urls_list

['https://www.gsmarena.com/samsung_galaxy_tab_s6_lite-10158.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_a_10_1_(2019)-9582.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_s6-9781.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_a_8_0_(2019)-9760.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_s5e-9581.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_a_10_1_(2016)-8090.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_s6_5g-10004.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_a_7_0_(2016)-7880.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_e_9_6-7295.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_a_8_0_&_s_pen_(2019)-9651.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_3_7_0-5422.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_s4_10_5-9262.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_a_10_5-9263.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_4_7_0-6251.php',
 'https://www.gsmarena.com/samsung_galaxy_tab_s2_9_7-7438.php',
 'https

In [29]:
len(all_tab_urls_list)

70

### Single Device Data Testing

In [30]:
driver.get(all_tab_urls_list[0])
time.sleep(10)

In [31]:
tab_data=BeautifulSoup(driver.page_source,'html.parser')

In [32]:
title=tab_data.find(class_ ="specs-phone-name-title").string
title

'Samsung Galaxy Tab S6 Lite'

In [33]:
key=title[8:].replace(' ', '-').replace('(', '').replace(')', '').replace('.', '_').lower()
key

'galaxy-tab-s6-lite'

In [34]:
val = tab_data.find_all("span",attrs= {"data-spec":True})


In [35]:
cam = tab_data.find_all("span",attrs={"data-spec": lambda x: x == "camerapixels-hl"})[0].string + ' MP'
cam

'8 MP'

In [36]:
storage = tab_data.find_all("span",attrs={"data-spec": lambda x: x == "storage-hl"})[0].string.split()[0]
storage

'64GB/128GB'

In [37]:
ram = tab_data.find_all("span",attrs={"data-spec": lambda x: x == "ramsize-hl"})[0].string
ram

'4'

In [38]:
chipset = tab_data.find_all("div",attrs={"data-spec": lambda x: x == "chipset-hl"})[0].string
chipset

'Exynos 9611'

In [39]:
img=tab_data.find(class_ ="specs-photo-main").img['src']
img

'https://fdn2.gsmarena.com/vv/bigpic/galaxy-tab-s6-lite.jpg'

In [40]:
img_name=title[8:].replace(' ', '-').replace('.', '_').strip().lower() +'.png'
img_name

'galaxy-tab-s6-lite.png'

In [41]:
opener = urllib.request.URLopener()
opener.addheader('User-Agent', 'whatever')
filename, headers = opener.retrieve(img, img_name)

## All Values

In [14]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Fetch the service account key JSON file contents
cred = credentials.Certificate('serviceAccountKey.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://personal-portfolio-12c66.firebaseio.com/'
})

In [15]:
# As an admin, the app has access to read and write all data, regradless of Security Rules
ref = db.reference('tablets/samsung/products')

In [17]:
## Main Loop
p=0
for i in all_tab_urls_list:
    print(i)
    p=p+1
    driver.get(i)
    time.sleep(10)
    
    tab_data = BeautifulSoup(driver.page_source,'html.parser')
    
    title = tab_data.find(class_ ="specs-phone-name-title").string
    # title
    
    key = title[8:].replace(' ', '-').replace('(', '').replace(')', '').replace('.', '_').lower()
    # key
    
    camera = tab_data.find_all("span",attrs={"data-spec": lambda x: x == "camerapixels-hl"})[0].string + ' MP'
    # camera

    storage = tab_data.find_all("span",attrs={"data-spec": lambda x: x == "storage-hl"})[0].string.split()[0]
    # storage
    
    ram = tab_data.find_all("span",attrs={"data-spec": lambda x: x == "ramsize-hl"})[0].string
    #ram
    
    chipset = tab_data.find_all("div",attrs={"data-spec": lambda x: x == "chipset-hl"})[0].string
    # chipset
    
    img = tab_data.find(class_ ="specs-photo-main").img['src']
    # img
    img_nm = title[8:].replace(' ', '-').replace('.', '_').strip().lower() +'.png'
    # img_nm
    
    time.sleep(10)

    opener = urllib.request.URLopener()
    opener.addheader('User-Agent', 'whatever')
    filename, headers = opener.retrieve(img, img_nm)
    
    image_path="tab/"+img_nm
    #image_path

    ref.child(key).set({
        'p_img': image_path,
        'p_name': title,
        'priority': p,
        'specification':{
        'camera':camera,
        'chipset':chipset,
        'storage':storage,
        'ram':ram
    }
    
    })


https://www.gsmarena.com/samsung_galaxy_tab_s6_lite-10158.php
https://www.gsmarena.com/samsung_galaxy_tab_a_10_1_(2019)-9582.php
https://www.gsmarena.com/samsung_galaxy_tab_s6-9781.php
https://www.gsmarena.com/samsung_galaxy_tab_a_8_0_(2019)-9760.php
https://www.gsmarena.com/samsung_galaxy_tab_s5e-9581.php
https://www.gsmarena.com/samsung_galaxy_tab_a_10_1_(2016)-8090.php
https://www.gsmarena.com/samsung_galaxy_tab_s6_5g-10004.php
https://www.gsmarena.com/samsung_galaxy_tab_a_7_0_(2016)-7880.php
https://www.gsmarena.com/samsung_galaxy_tab_e_9_6-7295.php
https://www.gsmarena.com/samsung_galaxy_tab_a_8_0_&_s_pen_(2019)-9651.php
https://www.gsmarena.com/samsung_galaxy_tab_3_7_0-5422.php
https://www.gsmarena.com/samsung_galaxy_tab_s4_10_5-9262.php
https://www.gsmarena.com/samsung_galaxy_tab_a_10_5-9263.php
https://www.gsmarena.com/samsung_galaxy_tab_4_7_0-6251.php
https://www.gsmarena.com/samsung_galaxy_tab_s2_9_7-7438.php
https://www.gsmarena.com/samsung_galaxy_tab_s2_8_0-7439.php
https:/